In [21]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import pandas as pd

In [22]:
service = webdriver.ChromeService("C:\chromedriver\chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.maximize_window()

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\DELL\AppData\Local\Temp\ipykernel_13260\3387883428.py:1: SyntaxWarning: invalid escape sequence '\c'
  service = webdriver.ChromeService("C:\chromedriver\chromedriver.exe")


In [23]:
check_num_products = True
num_pages = 0
num_products = 2000
columns, feature_name_cars, feature_value_cars = [], [], []

while check_num_products:
    if num_pages == 0:
        driver.get('https://carvago.com/cars')
    else: driver.get(f'https://carvago.com/cars?page={num_pages}')
    #driver.implicitly_wait(5)
    
    page_source = BeautifulSoup(driver.page_source, 'html.parser')
    link_items = page_source.find_all(class_='gtm-element-visibility-impressions-list css-10bdx8l')

    link_items = ['https://carvago.com' + link.get('href') for link in link_items]
    
    for i in range(len(link_items)):
        driver.get(link_items[i])
        page_source = BeautifulSoup(driver.page_source, 'html.parser')
        features = page_source.find_all(class_='css-1b3bvbl')
        feature_names, feature_values = [], []
        for j in range(len(features)):
            count_fea = 0
            for k in features[j].find_all('div'):
                if k.text:
                    count_fea += 1
                    if count_fea == 1:
                        fea_name = k.text
                    elif count_fea == 2:
                        fea_value = k.text
                        break
            feature_names.append(fea_name)
            feature_values.append(fea_value)
            if fea_name not in columns:
                columns.append(fea_name)
        
        feature_names.append('Price')
        price = int(''.join(page_source.find_all(class_='css-1ejrz84')[0].text.split('\xa0')[:2]))
        feature_values.append(price)
        
        feature_name_cars.append(feature_names)
        feature_value_cars.append(feature_values)
        
        print(f'Car {len(feature_name_cars)}: {feature_value_cars[i]}')
        
        if len(feature_name_cars) == num_products:
            check_num_products = False
            break
    num_pages += 1
        

Car 1: ['Fiat', '500X', 'Blue', 'Other interior color', 'Full leather interior', 'SUV / offroad', '4/5 doors', '5', 'not published by the seller', 'Petrol', 'Automatic', '4x2', '111 kW', '1\xa0332 ccm', '140 g/km', 'Euro 6d', '14\xa0117 km', '2/2022', '2', 28899]
Car 2: ['Ford', 'Focus', 'Black', 'Metallic', 'Other interior color', 'Cloth interior', 'Station Wagon', '4/5 doors', '5', '1395', 'not published by the seller', 'Diesel', 'Manual', '4x2', '88 kW', '1\xa0498 ccm', '98 g/km', 'No emission class', '85\xa0400 km', '6/2020', 'Used', 'Germany', 18799]
Car 3: ['Peugeot', '208', 'White', 'Hatchback', '4/5 doors', '5', 'not published by the seller', 'Petrol', 'Manual', '4x2', '74 kW', '1\xa0199 ccm', '97 g/km', 'Euro 6', '54\xa0030 km', '6/2020', 'Used', '1', '12/2024', 18599]
Car 4: ['Volkswagen', 'T-Roc', 'White', 'Grey interior', 'Cloth interior', 'Cabriolet', '4/5 doors', '4', 'not published by the seller', 'Petrol', 'Automatic', '4x2', '110 kW', '1\xa0498 ccm', '130 g/km', 'Euro 

In [25]:
columns.append('Price')
rows = [['' for _ in range(len(columns))] for _ in range(num_products)]
for i in range(num_products):
    for j in range(len(feature_name_cars[i])):
        if feature_value_cars[i][j]:
            rows[i][columns.index(feature_name_cars[i][j])] = feature_value_cars[i][j]

df = pd.DataFrame(rows, columns=columns)
df

,Make,Model,Body color,Interior color,Interior material,Body,Doors,Seats,VIN,Fuel,...,Battery capacity,Range extender,Hybrid type,AC charging time,DC charging time,Electric range,Secondary drive,Energy efficiency,Full service history,Price
0,Fiat,500X,Blue,Other interior color,Full leather interior,SUV / offroad,4/5 doors,5,not published by the seller,Petrol,...,,,,,,,,,,28899
1,Ford,Focus,Black,Other interior color,Cloth interior,Station Wagon,4/5 doors,5,not published by the seller,Diesel,...,,,,,,,,,,18799
2,Peugeot,208,White,,,Hatchback,4/5 doors,5,not published by the seller,Petrol,...,,,,,,,,,,18599
3,Volkswagen,T-Roc,White,Grey interior,Cloth interior,Cabriolet,4/5 doors,4,not published by the seller,Petrol,...,,,,,,,,,,42649
4,Jeep,Wrangler,Black,Black interior,Full leather interior,SUV / offroad,4/5 doors,5,not published by the seller,Petrol,...,,,,,,,,,,59999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Mercedes-Benz,C 180,Silver,Black interior,Cloth interior,Station Wagon,4/5 doors,5,not published by the seller,Petrol,...,,,,,,,,,,39149
1996,Audi,A3,Black,Black interior,Part leather interior,Sedans / saloons,4/5 doors,5,not published by the seller,Petrol,...,,,,,,,,,,29899
1997,Cupra,Formentor,White,Black interior,Cloth interior,SUV / offroad,4/5 doors,5,not published by the seller,Petrol,...,,,,,,,,,,40799
1998,Mercedes-Benz,GLE 400,Black,Black interior,Full leather interior,SUV / offroad,4/5 doors,5,not published by the seller,Petrol,...,,,,,,,,,,47399


In [26]:
df.to_csv('carvago.csv') # là raw_data.csv